Query Function shorter

In [6]:
def query_short(service, site_url, payload):
    """Run a query on the Google Search Console API and return a dataframe of results.
    
    Args:
        service (object): Service object from connect()
        site_url (string): URL of Google Search Console property
        payload (dict): API query payload dictionary
    
    Return:
        df (dataframe): Pandas dataframe containing requested data. 
    
    """
    
    response = service.searchanalytics().query(siteUrl=site_url, body=payload).execute()
    
    results = []
    
    for row in response['rows']:    
        data = {}
        
        for i in range(len(payload['dimensions'])):
            data[payload['dimensions'][i]] = row['keys'][i]

        data['clicks'] = row['clicks']
        data['impressions'] = row['impressions']
        results.append(data)
    
    return pd.DataFrame.from_dict(results)

In [523]:
# Google Search datetime is in California time, convert to local Stockholm time. NOT USED

from datetime import datetime
import pendulum  # $ pip install pendulum

# Basic datetime creation
todayLocal = pendulum.now("Europe/Stockholm")
print(todayLocal) 
print(isinstance(todayLocal, datetime))  # True  (pendulum datetime inherits from built-in datetime)
print(todayLocal.timezone.name)  # UTC (By default datetimes are stored in UTC timezone)

# Convert to PST
todayPST = todayLocal.in_timezone("America/Los_Angeles")
print(todayPST) 
print(isinstance(todayPST, datetime))  # True  (pendulum datetime inherits from built-in datetime)
print(todayPST.timezone.name)  # UTC (By default datetimes are stored in UTC timezone)

# 10 days ago
timeStart = todayPST.add(days=-9)
# 3 days ago
timeEnd = todayPST.add(days=-3)

print(timeStart)
print(timeEnd)

# To string only date
timeStart = timeStart.strftime('%Y-%m-%d')
timeEnd = timeEnd.strftime('%Y-%m-%d')

print(timeStart)
print(timeEnd)


ImportError: cannot import name 'strftime' from 'datetime' (/Users/emil/miniforge3/envs/googleapi/lib/python3.11/datetime.py)

In [308]:
# Google Search datetime is in California time, convert to local Stockholm time. 
# Date will be in PST since this is what google search expects. Using 12 PM PST for search
# In analytics this represents around 9 pm Swedish time, so should get data for the same full day.

# Enter relative date
start_date = 'one week ago'
end_date = 'today 12:00 pm'

print(start_date)
print(end_date)

start_date    = dateparser.parse(start_date, settings={'TIMEZONE': 'America/Los_Angeles'})
print(start_date)
#start_date     = start_date.strftime("%Y-%m-%d")

end_date   = dateparser.parse(end_date, settings={'TIMEZONE': 'America/Los_Angeles'})
print(end_date)
#end_date     = end_date.strftime("%Y-%m-%d")

print(start_date)
print(end_date)

one week ago
today 12:00 pm
2023-05-09 08:50:18.492604
2023-05-16 12:00:00
2023-05-09 08:50:18.492604
2023-05-16 12:00:00


In [309]:
# TESTA to get the date for query, just want to query one day at a time, LOS ANGELS TIME ZONE
test_date = '2022-08-01 12 pm'
test_date2 = 'one week ago'

testdateUS = dateparser.parse(test_date, settings={'TIMEZONE': 'America/Los_Angeles'})
testdateSWE = dateparser.parse(test_date, settings={'TIMEZONE': 'Europe/Stockholm'})

testdate2US = dateparser.parse(test_date2, settings={'TIMEZONE': 'America/Los_Angeles'})
testdate2SWE = dateparser.parse(test_date2, settings={'TIMEZONE': 'Europe/Stockholm'})

print(testdateUS)
print(testdateSWE)
print('....................')
print(testdate2US)
print(testdate2SWE)


2022-08-01 12:00:00
2022-08-01 12:00:00
....................
2023-05-09 08:50:20.543076
2023-05-09 17:50:20.546116


In [208]:
# THIS IS JUST TO READ IN AN EARLIER FILE WHICH HAVE EXTRA INDEX AND REMOVE IT
# THAT shold not be needed with new df where that should be sorted as above.
#df_old = pd.read_csv("output/csv/test2.csv")
df_new = pd.read_csv("output/csv/test2-4.csv", index_col=0)
df_new.reset_index(inplace=True, drop=True)
df_new.set_index('date', inplace=True)
df_new.head()

,Unnamed: 0,page,device,query,country,clicks,impressions,ctr,position
date,,,,,,,,,
2022-01-10,0.0,https://advokatfamiljforsvar.se/lvu-advokat-i-...,DESKTOP,lvu västerås,twn,0,1,0.0,25.0
2022-01-10,1.0,https://advokatfamiljforsvar.se/tag/gratis-rad...,DESKTOP,advokat delad vårdnad göteborg,swe,0,1,0.0,57.0
2022-01-10,2.0,https://advokatfamiljforsvar.se/tag/gratis-rad...,DESKTOP,advokat vårdnadstvist malmö,swe,0,1,0.0,71.0
2022-01-10,3.0,https://advokatfamiljforsvar.se/tag/gratis-rad...,DESKTOP,bästa advokat familjerätt stockholm,irn,0,1,0.0,92.0
2022-01-10,4.0,https://advokatfamiljforsvar.se/tag/gratis-rad...,DESKTOP,gratis rådgivning familjerätt,tha,0,1,0.0,87.0


In [210]:
#df_new.rename({"Unnamed: 0":"droppa"}, axis="columns", inplace=True)
df_new.drop(["droppa"], axis=1, inplace=True)
df_new.head()
#df_new.to_csv('/Users/emil/Downloads/ORIGINALTEST.csv', index='date')  

,page,device,query,country,clicks,impressions,ctr,position
date,,,,,,,,
2022-01-10,https://advokatfamiljforsvar.se/lvu-advokat-i-...,DESKTOP,lvu västerås,twn,0,1,0.0,25.0
2022-01-10,https://advokatfamiljforsvar.se/tag/gratis-rad...,DESKTOP,advokat delad vårdnad göteborg,swe,0,1,0.0,57.0
2022-01-10,https://advokatfamiljforsvar.se/tag/gratis-rad...,DESKTOP,advokat vårdnadstvist malmö,swe,0,1,0.0,71.0
2022-01-10,https://advokatfamiljforsvar.se/tag/gratis-rad...,DESKTOP,bästa advokat familjerätt stockholm,irn,0,1,0.0,92.0
2022-01-10,https://advokatfamiljforsvar.se/tag/gratis-rad...,DESKTOP,gratis rådgivning familjerätt,tha,0,1,0.0,87.0


In [65]:
# SHOULD NOT BE NEEDED WITH FUNCTION ABOVE

# Read in old df
df_old = pd.read_csv("output/csv/test2.csv")

# Add newly created df to old readin df
final_df = pd.concat([df_old, df_new], ignore_index=True)

# Sort
final_df.sort_values(by=["date","page","device","query","country"],ascending=True,inplace=True)


FileNotFoundError: [Errno 2] No such file or directory: 'output/csv/test2.csv'

TEST QUERIES

In [555]:

payload = {
    'startDate': start_date,
    'endDate': end_date,
    'dimensions': ["date","page","device","query","country"],
    'rowLimit': 25000,
    'startRow': 0
}

site_url = "https://advokatfamiljforsvar.se"

df = query_full(service, site_url, payload)
df.sort_values(by=["date","page","device","query","country"],ascending=True,inplace=True)
force_show_all(df)

TypeError: Object of type datetime is not JSON serializable

In [395]:

payload = {
    'startDate': start_date,
    'endDate': end_date,
    'dimensions': ["date","page","device","query","country"],
    'rowLimit': 100,
    'startRow': 0
}

site_url = "https://advokatfamiljforsvar.se"

df = query_full(service, site_url, payload)
df.sort_values(by=[],ascending=True,inplace=True)
force_show_all(df)

,date,page,device,query,country,clicks,impressions,ctr,position
0,2023-05-05,https://advokatfamiljforsvar.se/,DESKTOP,familj och försvar,swe,7,7,100.00,1.00
1,2023-05-05,https://advokatfamiljforsvar.se/vad-ar-barnfridsbrott/,MOBILE,barnfridsbrott,swe,5,73,6.85,4.00
2,2023-05-05,https://advokatfamiljforsvar.se/narkotikabrott-och-straff-ringa-normalgrad-och-grovt-brott/,MOBILE,narkotikabrott straff,swe,4,24,16.67,1.00
3,2023-05-05,https://advokatfamiljforsvar.se/narkotikabrott-och-straff-ringa-normalgrad-och-grovt-brott/,MOBILE,straffskala narkotika,swe,4,13,30.77,1.00
4,2023-05-06,https://advokatfamiljforsvar.se/narkotikabrott-och-straff-ringa-normalgrad-och-grovt-brott/,DESKTOP,ringa narkotikabrott straff,swe,3,6,50.00,1.00
5,2023-05-06,https://advokatfamiljforsvar.se/narkotikabrott-och-straff-ringa-normalgrad-och-grovt-brott/,MOBILE,ringa narkotikabrott straff,swe,3,11,27.27,1.45
6,2023-05-06,https://advokatfamiljforsvar.se/narkotikabrott-och-straff-ringa-normalgrad-och-grovt-brott/,MOBILE,straffskala narkotika,swe,3,8,37.50,1.00
7,2023-05-07,https://advokatfamiljforsvar.se/narkotikabrott-och-straff-ringa-normalgrad-och-grovt-brott/,MOBILE,narkotikabrott straff,swe,3,19,15.79,1.00
8,2023-05-07,https://advokatfamiljforsvar.se/vad-ar-barnfridsbrott/,MOBILE,barnfridsbrott,swe,3,24,12.50,4.00
9,2023-05-04,https://advokatfamiljforsvar.se/informationssamtal-innan-vardnadstvist-samtal-hos-familjeratten/,MOBILE,informationssamtal,swe,2,7,28.57,5.14


In [51]:

payload = {
    'startDate': "2023-01-01",
    'endDate': "2023-02-31",
    'dimensions': ["page","device"],
    'rowLimit': 100,
    'startRow': 0
}

site_url = "https://advokatfamiljforsvar.se"

df = query_full(service, site_url, payload)
df.head()

,page,device,clicks,impressions,ctr,position
0,https://advokatfamiljforsvar.se/vad-ar-barnfri...,MOBILE,220,4238,5.19,5.65
1,https://advokatfamiljforsvar.se/maste-jag-ga-p...,MOBILE,186,5172,3.60,6.11
2,https://advokatfamiljforsvar.se/vad-ar-barnfri...,DESKTOP,177,4159,4.26,8.76
3,https://advokatfamiljforsvar.se/,DESKTOP,60,2167,2.77,50.97
4,https://advokatfamiljforsvar.se/familjeratt/un...,MOBILE,53,3314,1.60,17.63


In [52]:

payload = {
    'startDate': "2023-01-01",
    'endDate': "2023-02-31",
    'dimensions': ["page","country"],
    'rowLimit': 100,
    'startRow': 0
}

site_url = "https://advokatfamiljforsvar.se"

df = query_full(service, site_url, payload)
df.head()

,page,country,clicks,impressions,ctr,position
0,https://advokatfamiljforsvar.se/vad-ar-barnfri...,swe,390,7833,4.98,4.87
1,https://advokatfamiljforsvar.se/maste-jag-ga-p...,swe,221,6241,3.54,5.62
2,https://advokatfamiljforsvar.se/,swe,109,715,15.24,20.09
3,https://advokatfamiljforsvar.se/om-oss/om-advo...,swe,72,1593,4.52,14.93
4,https://advokatfamiljforsvar.se/familjeratt/un...,swe,70,3101,2.26,14.74


In [53]:

payload = {
  "startDate": "2023-01-01",
  "endDate": "2023-02-31",
  "type": "web",
  "dimensions": [
    "searchAppearance"
  ]
}

site_url = "https://advokatfamiljforsvar.se"

df = query_full(service, site_url, payload)
df.head()

,searchAppearance,clicks,impressions,ctr,position
0,PAGE_EXPERIENCE,1673,87840,1.90,25.67
1,TPF_FAQ,1065,53971,1.97,17.78
2,VIDEO,9,213,4.23,10.86


In [66]:

payload = {
    'startDate': "2023-01-01",
    'endDate': "2023-01-01",
    'dimensions': ["page"],
    "aggregationType": "byPage",
}

site_url = "https://advokatfamiljforsvar.se"

df = query_full(service, site_url, payload)
force_show_all(df)

,page,clicks,impressions,ctr,position
0,https://advokatfamiljforsvar.se/maste-jag-ga-pa-polisforhor/,11,198,5.56,5.72
1,https://advokatfamiljforsvar.se/custody-disputes-how-to-apply-for-sole-custody-in-sweden-vardnadstvist/,4,51,7.84,40.29
2,https://advokatfamiljforsvar.se/familjeratt/lvu-och-tvangsvard/lvu-nar-barn-omhandertas/,3,73,4.11,22.14
3,https://advokatfamiljforsvar.se/vad-ar-barnfridsbrott/,3,111,2.70,8.03
4,https://advokatfamiljforsvar.se/familjeratt/underhallsbidrag-till-barn-och-maka/underhallsbidrag-till-maka/,2,17,11.76,5.47
5,https://advokatfamiljforsvar.se/familjeratt/vardnad-boende-och-umgange/barns-boende-efter-separation-eller-skilsmassa/,2,41,4.88,25.54
6,https://advokatfamiljforsvar.se/familjeratt/vardnad-boende-och-umgange/ungange-med-barn/,2,43,4.65,11.70
7,https://advokatfamiljforsvar.se/var-ska-barnen-bo-efter-separation-eller-skilsmassa/,2,120,1.67,25.02
8,https://advokatfamiljforsvar.se/vardnadstvist-att-ansoka-om-ensam-vardnad/,2,108,1.85,21.54
9,https://advokatfamiljforsvar.se/familjeratt/lvu-och-tvangsvard/forvaltarskap-och-god-man/,1,19,5.26,24.26


MASSA KODURKLIPP, FOR TEST, ANVANDS INTE

In [ ]:
df = pd.DataFrame(columns=["page", "clicks", "impressions",	"ctr", "position"])

entry = pd.DataFrame.from_dict({
     "firstname": ["John"],
     "lastname":  ["Johny"]
})

df = pd.concat([df, entry], ignore_index=True)


int maxRows = 25000; // Current max response size
int i = 0;
do {
  response = Request(startDate = 3_days_ago,
                     endDate = 3_days_ago,
                     ... add dimensions, type ...
                     rowLimit = maxRows,
                     startRow = i * maxRows);
  i++;
  …  // Do something with the response data.
} while (response.rows.count() != 0); // Page through all result rows



payload = {
    'startDate': "2023-01-01",
    'endDate': "2023-01-02",
    'dimensions': ["page"],
    "aggregationType": "byPage",
}

site_url = "https://advokatfamiljforsvar.se"

df = query(service, site_url, payload)

df = pd.concat([df, entry], ignore_index=True)

force_show_all(df)


df_row_merged = pd.concat([df_a, df_b], ignore_index=True)
df_col_merged = pd.concat([df_a, df_b], axis=1)

frames = [df_A, df_B]  # Or perform operations on the DFs
result = pd.concat(frames)

TESTING HOW TO JOIN SEVERAL QUERIES AND COMBINE INTO ONE DF

In [231]:
payload = {
    'startDate': "2023-01-01",
    'endDate': "2023-01-02",
    'dimensions': ["page"],
    "aggregationType": "byPage",
}

dfbase = query_short(service, site_url, payload)
dfadd.fillna(0, inplace=True)
dfbase = dfbase.replace('NaN', 0)
dfbase.replace(np.NaN, 0, inplace=True)

In [232]:

payload = {
    'startDate': "2023-01-03",
    'endDate': "2023-01-04",
    'dimensions': ["page"],
    "aggregationType": "byPage",
}

site_url = "https://advokatfamiljforsvar.se"

dfadd = query_short(service, site_url, payload)
dfadd.fillna(0, inplace=True)
dfadd = dfadd.replace('NaN', 0)
dfadd.replace(np.NaN, 0, inplace=True)

dfbase = dfbase.merge(dfadd, how='outer', on='page')
#dfbase= dfbase.merge(dfadd, left_on='page', right_on='page')
#dfbase = pd.concat([dfbase, dfadd], ignore_index=True)

In [233]:

payload = {
    'startDate': "2023-01-05",
    'endDate': "2023-01-06",
    'dimensions': ["page"],
    "aggregationType": "byPage",
}

site_url = "https://advokatfamiljforsvar.se"

dfadd = query_short(service, site_url, payload)
dfadd.fillna(0, inplace=True)
dfadd = dfadd.replace('NaN', 0)
dfadd.replace(np.NaN, 0, inplace=True)

dfbase = dfbase.merge(dfadd, how='outer', on='page')
#dfbase= dfbase.merge(dfadd, left_on='page', right_on='page')
#dfbase = pd.concat([dfbase, dfadd], ignore_index=True)

In [234]:
dfbase.fillna(0, inplace=True)
dfbase = dfbase.replace('NaN', 0)
dfbase.replace(np.NaN, 0, inplace=True)

In [235]:
# Sum Columns together 
dfbase['clicks'] = dfbase.sum(axis=1, numeric_only=True)

In [236]:
#Copy of dfbase, with only sum selected as integer

dfPageClicks = (dfbase.iloc[ : , [0,-1]]).copy()

dfPageClicks['clicks'] = dfPageClicks['clicks'].astype(int)
dfPageClicks.sort_values(by=['clicks'],ascending=False,inplace=True)


force_show_all(dfPageClicks)

,page,clicks
0,https://advokatfamiljforsvar.se/maste-jag-ga-pa-polisforhor/,68
1,https://advokatfamiljforsvar.se/vad-ar-barnfridsbrott/,34
2,https://advokatfamiljforsvar.se/custody-disputes-how-to-apply-for-sole-custody-in-sweden-vardnadstvist/,14
3,https://advokatfamiljforsvar.se/familjeratt/lvu-och-tvangsvard/lvu-nar-barn-omhandertas/,12
4,https://advokatfamiljforsvar.se/var-ska-barnen-bo-efter-separation-eller-skilsmassa/,12
5,https://advokatfamiljforsvar.se/vardnadstvist-att-ansoka-om-ensam-vardnad/,12
10,https://advokatfamiljforsvar.se/om-oss/om-advokat-johan-lindgren/,10
6,https://advokatfamiljforsvar.se/familjeratt/underhallsbidrag-till-barn-och-maka/underhallsbidrag-till-maka/,10
7,https://advokatfamiljforsvar.se/familjeratt/skilsmassa-och-bodelning/bostaden-och-kvarsittningsratt-vid-skilsmassa/,9
41,https://advokatfamiljforsvar.se/familjeratt/underhallsbidrag-till-barn-och-maka/underhallsbidrag-till-barn/,8


TEST TO SEE IF MERGING DF FROM DIFFERENT DATES WORK

In [569]:
#df4 = df2.set_index(['page', 'device', 'query', 'country'])
df4 = df2.set_index(['page'])

#df5 = df3.set_index(['page', 'device', 'query', 'country'])
df5 = df3.set_index(['page'])

df6 = df4 + df5
df8 = df2 + df3

df7 = pd.concat([df2, df3]).groupby(by=['page', 'device', 'query', 'country']).sum()

#df6.replace(np.NaN, 0, inplace=True)

#df6.sort_values(by=['page'],ascending=False,inplace=True)

#df6 = df6.set_index(['page'])

print(df7)

                                                                                                            date  clicks  impressions
page                                               device  query                             country                                 
https://advokatfamiljforsvar.se/                   DESKTOP advokat                           gbr      2023-05-04       0            1
                                                                                             ind      2023-05-04       0            1
                                                                                             kor      2023-05-04       0            2
                                                                                             mys      2023-05-04       0            1
                                                                                             pse      2023-05-04       0            1
...                                                           

Save to CSV

In [570]:
os.makedirs('output/csv', exist_ok=True)  
df7.to_csv('output/csv/test2.csv')  

In [ ]:

def query_full(service, site_url, payload):
    """Run a query on the Google Search Console API and return a dataframe of results.
    
    Args:
        service (object): Service object from connect()
        site_url (string): URL of Google Search Console property
        payload (dict): API query payload dictionary
    
    Return:
        df (dataframe): Pandas dataframe containing requested data. 
    
    """
    
    response = service.searchanalytics().query(siteUrl=site_url, body=payload).execute()
    
    results = []
    
    for row in response['rows']:    
        data = {}
        
        for i in range(len(payload['dimensions'])):
            data[payload['dimensions'][i]] = row['keys'][i]

        data['clicks'] = row['clicks']
        data['impressions'] = row['impressions']
        data['ctr'] = round(row['ctr'] * 100, 2)
        data['position'] = round(row['position'], 2)        
        results.append(data)
    
    return pd.DataFrame.from_dict(results)


In [454]:
# Loop to send query between start_date and end_date-1 day. Each date to a new DF in dfs. 
# Then concatenated into final DF.

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

# Only run if dates are not already in old csv.        
        
if test_start_dateDT > maxSavedDateDT and three_days_agoDT > end_dateDT and three_days_agoDT > maxSavedDateDT:
        
    start_date = test_start_dateDT
    end_date = test_end_dateDT

    dfs = []

    for query_date in daterange(start_date, end_date):
        query_date = query_date.strftime("%Y-%m-%d")

        payload = {
            'startDate': query_date,
            'endDate': query_date,
            'dimensions': ["date","page","device","query","country"],
            'rowLimit': 25000,
            'startRow': 0
            }

        site_url = "https://advokatfamiljforsvar.se"

        try:
        
            df = query_full(service, site_url, payload)
            dfs.append(df)

        except KeyError:

            break

        # New DF med all data fran start_date - end_data-1 
        df_new = pd.concat(dfs, ignore_index=True)
        df_new.reset_index(inplace=True, drop=True)
            
        # Sort columns in same order as the file to write to
        df_new.sort_values(by=["date","page","device","query","country"],ascending=True,inplace=True)
            
        # Columns to case matching reference and analytics data
        df_new['device'] = df_new['device'].str.lower()
        df_new['country'] = df_new['country'].str.upper()
            
        # Column names to matching reference and analytics data
        df_new.rename(columns = {'date':'date','country':'countryIdLong', 'page':'landingPage', 'device':'deviceCategory'}, inplace = True)
            
        # To File
        csvFilePath = "output/csv/"
        fileName = "testa.csv"
        sep = ","
            
        dates_funcs.appendDFToCSV(df, csvFilePath, fileName, sep)
     
else:
    
    print('Wait a day, before running again')
    #display(df_new.tail())


'\ndef daterange(start_date, end_date):\n    for n in range(int((end_date - start_date).days)):\n        yield start_date + timedelta(n)\n\n# Only run if dates are not already in old csv.        \n        \nif test_start_dateDT > maxSavedDateDT and three_days_agoDT > end_dateDT and three_days_agoDT > maxSavedDateDT:\n        \n    start_date = test_start_dateDT\n    end_date = test_end_dateDT\n\n    dfs = []\n\n    for query_date in daterange(start_date, end_date):\n        query_date = query_date.strftime("%Y-%m-%d")\n\n        payload = {\n            \'startDate\': query_date,\n            \'endDate\': query_date,\n            \'dimensions\': ["date","page","device","query","country"],\n            \'rowLimit\': 25000,\n            \'startRow\': 0\n            }\n\n        site_url = "https://advokatfamiljforsvar.se"\n\n        try:\n        \n            df = query_full(service, site_url, payload)\n            dfs.append(df)\n\n        except KeyError:\n\n            break\n\n     

In [ ]:

sa_request = {
        "startDate": start_date,
        "endDate": end_date,
        "dimensions": [
            "QUERY",
            "PAGE"
        ],
        "rowLimit": 25000,
    }



In [450]:
# Find maximum date in csv file.

# Read file in chunks (parts)
df = pd.read_csv("output/csv/testa.csv", chunksize=100000)

# Max date to start comparing to
#max_date_overall = testdateUS.strftime("%Y-%m-%d")
max_date_overall = datetime.datetime(2022, 1, 1).date()

#time.strptime(date1, "%d/%m/%Y")

print(max_date_overall)


for chunk in df:
    chunk['date'] = chunk['date']
    max_date_in_chunk = max(chunk['date'])
    
    max_date_in_chunk = datetime.datetime.strptime(max_date_in_chunk,'%Y-%m-%d').date()
    
    if max_date_in_chunk > max_date_overall:
        max_date_overall = max_date_in_chunk
    
    print('maxdate in chunk: ',max_date_overall) 

test_start_date_name = '2022-01-01'
test_start_date = dateparser.parse(test_start_date_name, settings={'TIMEZONE': 'America/Los_Angeles'}).date()

test_end_date_name = 'three days ago'
test_end_date = dateparser.parse(test_end_date_name, settings={'TIMEZONE': 'America/Los_Angeles'}).date()

print('maxdate in csv: ',max_date_overall) 
print('start date given: ',test_start_date)
print('end date given: ',test_end_date)

start_fetch_date = max_date_overall+datetime.timedelta(days=1)
start_fetch_date = start_fetch_date.strftime("%Y-%m-%d")

if test_start_date > max_date_overall:
    print('start date is after largest date in csv,', end='\n')
    print('will fetch updated data from date: ', start_fetch_date, end='\n')   
    print('to : ', test_end_date)
else:
    print('data already fetched, wait a day before running again')

print(max_date_overall+datetime.timedelta(days=1))

print(start_fetch_date)
print(test_end_date)


'\n# Read file in chunks (parts)\ndf = pd.read_csv("output/csv/testa.csv", chunksize=100000)\n\n# Max date to start comparing to\n#max_date_overall = testdateUS.strftime("%Y-%m-%d")\nmax_date_overall = datetime.datetime(2022, 1, 1).date()\n\n#time.strptime(date1, "%d/%m/%Y")\n\nprint(max_date_overall)\n\n\nfor chunk in df:\n    chunk[\'date\'] = chunk[\'date\']\n    max_date_in_chunk = max(chunk[\'date\'])\n    \n    max_date_in_chunk = datetime.datetime.strptime(max_date_in_chunk,\'%Y-%m-%d\').date()\n    \n    if max_date_in_chunk > max_date_overall:\n        max_date_overall = max_date_in_chunk\n    \n    print(\'maxdate in chunk: \',max_date_overall) \n\ntest_start_date_name = \'2022-01-01\'\ntest_start_date = dateparser.parse(test_start_date_name, settings={\'TIMEZONE\': \'America/Los_Angeles\'}).date()\n\ntest_end_date_name = \'three days ago\'\ntest_end_date = dateparser.parse(test_end_date_name, settings={\'TIMEZONE\': \'America/Los_Angeles\'}).date()\n\nprint(\'maxdate in csv: